In [1]:
from hybrid import ListCombinationHybrid
from Recommenders.KNN.ItemKNNCFRecommender import *
from Recommenders.KNN.UserKNNCFRecommender import *
from Data_Handler.DataReader import DataReader
from evaluator import evaluate
from tqdm import tqdm
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

dataReader = DataReader()

target = dataReader.load_target()
URM = dataReader.load_augmented_binary_urm()
URM_aug, ICM = dataReader.pad_with_zeros_ICMandURM(URM)

URM_train_aug, URM_validation = split_train_in_two_percentage_global_sample( URM_aug, train_percentage=0.9)

ItemKNNCF = ItemKNNCFRecommender(URM_train_aug)
ItemKNNCF.fit()

UserKNNCF = UserKNNCFRecommender(URM_train_aug)
UserKNNCF.fit()


recommender = ListCombinationHybrid(URM_train_aug,ItemKNNCF,UserKNNCF)
recommender.fit()

recommended_items_for_each_user = {}
for user_id in tqdm(target):
    recommended_items = recommender.recommend(
        user_id, cutoff=10, remove_seen_flag=True)
    recommended_items_for_each_user[int(user_id)] = recommended_items

# Evaluate recommended items
map = evaluate(recommended_items_for_each_user, URM_validation, target)
print('MAP score: {}'.format(map))


--->  23091
==> 24507
ItemKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Similarity column 27968 (100.0%), 4313.03 column/sec. Elapsed time 6.48 sec
UserKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Similarity column 41629 (100.0%), 1344.56 column/sec. Elapsed time 30.96 sec
ListCombinationHybrid: URM Detected 3461 (12.4%) items with no interactions.


100%|██████████| 41116/41116 [00:02<00:00, 15244.25it/s]

MAP score: 0.034229716072742056
